In [41]:
from Data import ParticleDataGenerator, ParticleDataset
from Model import ParticleClassifier
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

n = 1000
n_scalar = 7
energy_matrix_size = 7
high_res_size = 500 # energy high resolution size
generator = ParticleDataGenerator(num_data_points=n, num_scalars=n_scalar, energy_matrix_size=energy_matrix_size, high_res_size=high_res_size, seed=42)

# Generate the data
category_mean_diff = 0.5
std_dev_ratio = 1.5
category_ratio = 0.5
scalar_inputs, energy_distributions, labels = generator.generate_data(category_mean_diff, std_dev_ratio, category_ratio)

# Create the dataset
dataset = ParticleDataset(scalar_inputs, energy_distributions, labels)
train_size = int(0.8 * len(dataset))  # 假設使用80%的數據作為訓練集
test_size = len(dataset) - train_size

# 隨機分割數據集
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create a DataLoader instance
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# hyperparameter
learning_rate = 0.001
num_epochs = 50

model = ParticleClassifier(n_scalar)
criterion = nn.BCELoss()  # Binary classification loss
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

def train(model, train_loader, num_epochs, verbose=True):
    for epoch in range(num_epochs):
        total = 0
        correct = 0
        # for i, (scalars, images, labels) in enumerate(tqdm(train_loader)):
        for scalars, images, labels in train_loader:
            # forward 
            # in pytorch conv2D is CHW (channel, height, width)
            outputs = model(scalars, images.unsqueeze(1))  # 添加額外的維度作為通道數(channel)
            # print('outputs: ', outputs)
            loss = criterion(outputs[:, 1].unsqueeze(1), labels)

            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # predict, compute accuracy
            pred = torch.argmax(outputs, axis=1)
            # print('pred: ', pred)
            # print('pred shape: ', pred.unsqueeze(0).shape)
            # print('labels: ', labels)
            total += labels.size(0)
            correct += torch.sum(torch.eq(pred, labels.squeeze()))
        accuracy = correct / total * 100
        if verbose:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

def test(model, test_loader, verbose=True):
    model.eval() 
    total = 0
    correct = 0
    with torch.no_grad():
        for scalars, images, labels in test_loader:
            outputs = model(scalars, images.unsqueeze(1))
            pred = torch.argmax(outputs, axis=1)
            total += labels.size(0)
            correct += torch.sum(torch.eq(pred, labels.squeeze()))

    accuracy = 100 * correct / total
    if verbose:
        print(f'Test Accuracy: {accuracy:.2f}%')

    return accuracy / 100

train(model, train_loader, num_epochs,verbose=False)
test_acc = test(model, test_loader,verbose=False)
print('test_acc: {:.4f}'.format(test_acc.item()))

test_acc: 0.7900


In [43]:
# generate accuracy table
acc_table = []

n = 1000
n_scalar = 7
energy_matrix_size = 7
high_res_size = 500 # energy high resolution size
# generator = ParticleDataGenerator(num_data_points=n, num_scalars=n_scalar, energy_matrix_size=energy_matrix_size, high_res_size=high_res_size, seed=42)

# Generate the data
category_mean_diff = 0
std_dev_ratio = 1
category_ratio = 0.5
# scalar_inputs, energy_distributions, labels = generator.generate_data(category_mean_diff, std_dev_ratio, category_ratio)

# hyperparameter
learning_rate = 0.001
num_epochs = 30

model = ParticleClassifier(n_scalar)
criterion = nn.BCELoss()  # Binary classification loss
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for i in np.arange(0, 2.2, 0.2): # mean diff
    acc_table.append([])
    for j in np.arange(1, 3.2, 0.2): # std ratio
        category_mean_diff = i
        std_dev_ratio = j
        generator = ParticleDataGenerator(num_data_points=n, num_scalars=n_scalar, energy_matrix_size=energy_matrix_size, high_res_size=high_res_size, seed=42)
        scalar_inputs, energy_distributions, labels = generator.generate_data(category_mean_diff, std_dev_ratio, category_ratio)

        # Create the dataset
        dataset = ParticleDataset(scalar_inputs, energy_distributions, labels)
        train_size = int(0.8 * len(dataset))  # 假設使用80%的數據作為訓練集
        test_size = len(dataset) - train_size

        # 隨機分割數據集
        train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

        # Create a DataLoader instance
        batch_size = 64
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        
        model = ParticleClassifier(n_scalar)
        criterion = nn.BCELoss()  # Binary classification loss
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        train(model=model, train_loader=train_loader, num_epochs=num_epochs, verbose=False)
        test_acc = test(model=model, test_loader=test_loader, verbose=False)
        print('mean_diff: {:.1f} | std ratio: {:.1f} | acc: {:.4f}'.format(category_mean_diff,std_dev_ratio,test_acc.item()))

        acc_table[-1].append(test_acc.item())

mean_diff: 0.0 | std ratio: 1.0 | acc: 0.5600
mean_diff: 0.0 | std ratio: 1.2 | acc: 0.6100
mean_diff: 0.0 | std ratio: 1.4 | acc: 0.7200
mean_diff: 0.0 | std ratio: 1.6 | acc: 0.8000
mean_diff: 0.0 | std ratio: 1.8 | acc: 0.9050
mean_diff: 0.0 | std ratio: 2.0 | acc: 0.9300
mean_diff: 0.0 | std ratio: 2.2 | acc: 0.9500
mean_diff: 0.0 | std ratio: 2.4 | acc: 0.9500
mean_diff: 0.0 | std ratio: 2.6 | acc: 0.9600
mean_diff: 0.0 | std ratio: 2.8 | acc: 0.9700
mean_diff: 0.0 | std ratio: 3.0 | acc: 0.9700
mean_diff: 0.2 | std ratio: 1.0 | acc: 0.5850
mean_diff: 0.2 | std ratio: 1.2 | acc: 0.6100
mean_diff: 0.2 | std ratio: 1.4 | acc: 0.7250
mean_diff: 0.2 | std ratio: 1.6 | acc: 0.8050
mean_diff: 0.2 | std ratio: 1.8 | acc: 0.8550
mean_diff: 0.2 | std ratio: 2.0 | acc: 0.9250
mean_diff: 0.2 | std ratio: 2.2 | acc: 0.9450
mean_diff: 0.2 | std ratio: 2.4 | acc: 0.9600
mean_diff: 0.2 | std ratio: 2.6 | acc: 0.9650
mean_diff: 0.2 | std ratio: 2.8 | acc: 0.9750
mean_diff: 0.2 | std ratio: 3.0 | 

In [47]:
acc_table[-1]

[1.0,
 1.0,
 0.9950000047683716,
 0.9850000143051147,
 0.9800000190734863,
 0.9750000238418579,
 0.9750000238418579,
 0.9750000238418579,
 0.9750000238418579,
 0.9800000190734863,
 0.9800000190734863]

In [48]:
import pandas as pd

mean_diffs = np.arange(0, 2.2, 0.2)  # 不同的mean_diff
std_ratios = np.arange(1, 3.2, 0.2)  # 不同的std_ratio

# 創建DataFrame
df = pd.DataFrame(acc_table, columns=std_ratios, index=mean_diffs)

# 設置列和行的名稱
df.columns.name = 'std_ratio' 
df.index.name = 'mean_diff'

# 顯示表格
df

std_ratio,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.4,2.6,2.8,3.0
mean_diff,,,,,,,,,,,
0.0,0.560,0.610,0.720,0.800,0.905,0.930,0.950,0.950,0.960,0.970,0.970
0.2,0.585,0.610,0.725,0.805,0.855,0.925,0.945,0.960,0.965,0.975,0.975
0.4,0.695,0.710,0.770,0.810,0.850,0.910,0.945,0.960,0.970,0.975,0.985
0.6,0.790,0.765,0.780,0.825,0.865,0.895,0.915,0.960,0.960,0.975,0.980
0.8,0.860,0.830,0.830,0.850,0.870,0.875,0.905,0.945,0.960,0.970,0.980
1.0,0.900,0.885,0.880,0.890,0.895,0.895,0.900,0.910,0.955,0.965,0.975
1.2,0.945,0.920,0.915,0.905,0.915,0.910,0.920,0.920,0.935,0.945,0.970
1.4,0.965,0.950,0.945,0.930,0.925,0.925,0.920,0.940,0.950,0.950,0.965
1.6,0.980,0.975,0.960,0.955,0.955,0.945,0.950,0.955,0.950,0.960,0.970
